In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import os
import optuna
import glob
from src.helpers import unsupervised_trainer_factory

folder_path = os.getcwd() + "/data/csvs"
csv_files = glob.glob(os.path.join(folder_path, '*.csv'))

print("Setup Complete")

Setup Complete


/home/evanvance/miniconda3/envs/ids-ai/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dfs = []

for file in csv_files:
    filename = file.split("/")[-1]
    df = pd.read_csv(file)
    df["Source File"] = filename
    dfs.append(df)

captures = pd.concat(dfs)

In [3]:
captures.columns

Index([' Destination Port', ' Flow Duration', ' Total Fwd Packets',
       ' Total Backward Packets', 'Total Length of Fwd Packets',
       ' Total Length of Bwd Packets', ' Fwd Packet Length Max',
       ' Fwd Packet Length Min', ' Fwd Packet Length Mean',
       ' Fwd Packet Length Std', 'Bwd Packet Length Max',
       ' Bwd Packet Length Min', ' Bwd Packet Length Mean',
       ' Bwd Packet Length Std', 'Flow Bytes/s', ' Flow Packets/s',
       ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min',
       'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max',
       ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean', ' Bwd IAT Std',
       ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags', ' Bwd PSH Flags',
       ' Fwd URG Flags', ' Bwd URG Flags', ' Fwd Header Length',
       ' Bwd Header Length', 'Fwd Packets/s', ' Bwd Packets/s',
       ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean',
       ' Packet Length Std', ' Packet Length Variance', '

In [4]:
captures.rename(columns={name:name.strip() for name in captures.columns}, inplace=True)

In [5]:
captures.shape

(2830743, 80)

In [6]:

captures["Label"].unique()


array(['BENIGN', 'Web Attack � Brute Force', 'Web Attack � XSS',
       'Web Attack � Sql Injection', 'DDoS', 'DoS slowloris',
       'DoS Slowhttptest', 'DoS Hulk', 'DoS GoldenEye', 'Heartbleed',
       'FTP-Patator', 'SSH-Patator', 'Bot', 'Infiltration', 'PortScan'],
      dtype=object)

In [7]:
captures["Label"] = captures["Label"] == 'BENIGN'

In [8]:
le = LabelEncoder()

captures["Label"] = le.fit_transform(captures["Label"])

This Next Block is important to note. The Isolation forest cannot deal with infinity so it must be delt with.

In [9]:
captures = captures.replace([np.inf, -np.inf], np.nan)
captures.dropna(inplace=True)

In [10]:
captures.shape

(2827876, 80)

In [11]:
features = captures.columns.drop(["Label", "Source File"])

X = captures[features]
y = captures["Label"]

X_train, X_test, y_train, y_test = train_test_split(X,y, shuffle=True, train_size=.8, test_size=.2)

In [ ]:
isoforest_trainer = unsupervised_trainer_factory(
        IsolationForest, 
        X_train, y_train, X_test, y_test, 
        metric="auc_roc"
    )

def objective(trial, params):
    params["n_estimators"] = trial.suggest_int("n_estimators", 50, 1000)
    params["bootstrap"] = trial.suggest_categorical("bootstrap", [True, False])
    params["max_samples"] = trial.suggest_float("max_samples", .3, 1.0)

    return isoforest_trainer(**params)

iso_forest_params = {
    "random_state":0,
    "n_jobs":-1,
    "n_estimators":100,
    "max_samples": 1.0
}

In [24]:
iso_forest_study = optuna.create_study(directions=["maximize"], study_name="Isolation Forest Parameter Tuning")
iso_forest_study.optimize(lambda trial: objective(trial, iso_forest_params), n_trials=500)

[I 2024-11-22 11:49:58,375] A new study created in memory with name: Isolation Forest Parameter Tuning
[W 2024-11-22 11:53:47,933] Trial 0 failed with parameters: {'n_estimators': 931, 'bootstrap': True, 'max_samples': 0.39489936532291103} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/home/evanvance/miniconda3/envs/ids-ai/lib/python3.10/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_523779/3685968336.py", line 9, in <lambda>
    iso_forest_study.optimize(lambda trial: objective(trial, iso_forest_params), n_trials=5000)
  File "/tmp/ipykernel_523779/2434112706.py", line 6, in objective
    return isoforest_trainer(**params)
  File "/home/evanvance/Code/School/network-anomaly-ai/src/helpers.py", line 107, in trainer
    return eval_model(mdl, X_test, y_test)[metric]
  File "/home/evanvance/Code/School/network-anomaly-ai/src/helpers.py", line 90, in eval_model
   

KeyboardInterrupt: 

The OCSVM took about an hour to trian and did not perform half as well as the isolation forest.
{'auc': 0.996731548529342,
 'precision': 0.9997661037563736,
 'recall': 0.5087117966295345,
 'accuracy': 0.5148973166032238}

In [ ]:
# ocsvm_trainer = trainer_factory(OneClassSVM, X_train, y_train)

In [ ]:
# ocsvm, _ = ocsvm_trainer()

# eval_model(ocsvm, X_test, y_test)